In [1]:
pip install selenium


[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests


[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

import pandas as pd

from bs4 import BeautifulSoup
import requests

import re

# 크롤링

네이버 지도의 경우, 크롤링 자체를 막아뒀음. 출력이 아예 안 됨.
그러나 아래 전체로 보는 페이지(카톡으로 보낸 링크)에서는 리뷰 크롤링이 가능함.

따라서 우회(?)하는 방식을 사용함.

링크 중간에 '1098819515' 이렇게 숫자가 나오는데, 이게 카페/음식점의 고유번호임
이 고유번호만 바꿔주면 크롤링 가능한 페이지로 들어갈 수 있음.

문제는 강남역 주변의 카페의 고유번호를 크롤링해야 하는데, 네이버지도에서는 못함.
그래서 네이버에 '강남역 카페'를 검색해서 나오는 몇 개의 카페는 고유번호를 가져올 수 있음.

이걸 가져와서 하나씩 링크에 넣고 리뷰를 크롤링하는 방법을 사용함.

## 네이버 지도 말고 네이버에서

In [218]:
#크롬 드라이버에 url 주소 넣고 실행
url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EA%B0%95%EB%82%A8%EC%97%AD+%EC%B9%B4%ED%8E%98'
# 강남역 카페

# 크롬 브라우저 시작
driver = webdriver.Chrome('/Users/imsoyeong/Downloads/chromedriver_mac64/chromedriver.exe') 
driver.get(url)

# 페이지가 완전히 로딩되도록 1초동안 기다림
time.sleep(1)

webpage = requests.get(url)
soup = BeautifulSoup(webpage.content, "html.parser")

/var/folders/y3/_25c1lg15r71b9q06fr4j0dr0000gn/T/ipykernel_42116/2079816369.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/imsoyeong/Downloads/chromedriver_mac64/chromedriver.exe')


In [122]:
# 총 5페이지 하나씩 넘기기
cafe_num = []
cnt = 1
while True:
    # 정규표현식 패턴을 사용하여 'place/'와 '?' 사이의 값을 추출
    pattern = r'place/([^?]+)'
    
    # 스크롤 로딩을 위함
    while True:
        bh = driver.execute_script('return document.body.scrollHeight')
        ah = driver.execute_script('return document.body.scrollHeight')
        if ah == bh:
            break
        bh = ah
        
    time.sleep(1)
    # 추출
    infos = soup.select('.CHC5F > .tzwk0')
    for info in infos:
        info = str(info.select)
        try:
            match = re.search(pattern, info)
            extracted_value = match.group(1)
            cafe_num.append(int(extracted_value))
            
        except:
            pass
    
    driver.find_element('xpath', '//*[@id="place-main-section-root"]/section/div/div[5]/a[2]').click()
    cnt +=1
    
    if cnt == 5:
        break

In [119]:
# 리스트에서 중복된 값 제거
list(set(cafe_num))
cafe_num.extend([1483295656, 1186606414, 1754767885, 1081873068, 1478381424, 21082407, 1490841787])
cafe_num = list(set(cafe_num))
cafe_num

[]

### 고유번호 하나씩 넣어서 리뷰 크롤링
url = https://pcmap.place.naver.com/restaurant/[34015652]/review/visitor?shouldKeywordUnfold=true

In [220]:
cafe_name = []
comments = []
for num in cafe_num:
    url = 'https://pcmap.place.naver.com/restaurant/{}/review/visitor?shouldKeywordUnfold=true'.format(num)
    #크롬 드라이버에 url 주소 넣고 실행
    driver.get(url)
    time.sleep(2)
    
    # 카페이름
    name = driver.find_element(By.CSS_SELECTOR, '.Fc1rA').text
    
    # 스크롤 로딩을 위함
    while True:
        bh = driver.execute_script('return document.body.scrollHeight')
        time.sleep(2)
        ah = driver.execute_script('return document.body.scrollHeight')
        if ah == bh:
            break
        bh = ah
    
    while True:
        try:
            # 리뷰 크롤링
            reviews = driver.find_elements(By.CSS_SELECTOR, '.zPfVt')
            for re in reviews:
                cafe_name.append(name)
                comments.append(re.text)
                
            # 더보기 버튼 누르기
            driver.find_element('xpath', '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[3]/div[2]/a').click()
            time.sleep(1.5)
        except:
            break


In [222]:
df = pd.DataFrame({'cafe_name':cafe_name,
                   'comments':comments})

df

,cafe_name,comments
0,더달달,초코디저트에 진심인 강남역카페 ‘더달달’ 🍫더달달 초코 디저트 먹으면 다른 매장은 ...
1,더달달,"강남역카페 더달달 달달핑크빙수, 쇼콜라 등 초콜릿 디저트 퀄리티가 아주 좋았어요. ..."
2,더달달,강남에서 조용하고 넓은 카페를 찾는다면 추천할 곳이에요! 1층 뿐만 아니라 2층도 ...
3,더달달,딸기 우유 빙수 쫀맛탱이에요 딸기가 밑에까지 가득 들어있어서 먹어도 먹어도 딸기가 ...
4,더달달,"더달달 진짜 최고,,❤️ 엔틱한 분위기에 내부 인테리어도 너무 예쁘고 진짜 찐초코란..."
...,...,...
3249,더달달,빙수랑 와플 너무 맛있어요♡
3250,더달달,여자분들이 많은 곳은 찐 디저트맛집이라던데 여기네요..자리에 사람들 다 여자분이시거...
3251,더달달,종류도 다양하고 저렴해서 좋아요😊
3252,더달달,양이 많아요
